In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### data reading

In [0]:
df = spark.read.format("parquet").option("header", "true").load("abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/products")

df.display()

In [0]:
df.drop("_rescued_data").display()

### Create Function

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
create or replace function databricks_cata.bronze.discount_fun(p_price double) 
returns double
language sql
return p_price * 0.90



In [0]:
%sql
select product_id,price,databricks_cata.bronze.discount_fun(price) as discounted_price 
from
products

In [0]:
from pyspark.sql.functions import expr

df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_fun(price)"))

display(df)

In [0]:
%sql
create or replace function databricks_cata.bronze.upper_fun(p_brand string)
returns string
language python
as
$$
    return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_cata.bronze.upper_fun(brand) as brand_upper
from
products

In [0]:
new_df = df.drop("_rescued_data")
new_df.display()


In [0]:
new_df.drop("_rescued_data").write.format("delta").mode("overwrite").save("abfss://silver@rtgpadlsgen2.dfs.core.windows.net/products")

In [0]:
read=spark.read.format("delta").load("abfss://silver@rtgpadlsgen2.dfs.core.windows.net/products")
display(read)

In [0]:
%sql
create table if not exists databricks_cata.silver.products
using delta
location 'abfss://silver@rtgpadlsgen2.dfs.core.windows.net/products'

In [0]:
%sql
select * from databricks_cata.silver.products;